# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 7 2022 1:54PM,239015,19,DEX0006966,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
1,Jun 7 2022 1:48PM,239014,10,MET210958,"Methapharm, Inc.",Released
2,Jun 7 2022 1:48PM,239014,10,MET210960,"Methapharm, Inc.",Released
3,Jun 7 2022 1:48PM,239014,10,MET210963,"Methapharm, Inc.",Released
4,Jun 7 2022 1:48PM,239014,10,MET210968,"Methapharm, Inc.",Released
5,Jun 7 2022 1:48PM,239014,10,MET210977,"Methapharm, Inc.",Released
6,Jun 7 2022 1:48PM,239014,10,MET210985,"Methapharm, Inc.",Released
7,Jun 7 2022 1:48PM,239014,10,MET210986,"Methapharm, Inc.",Released
8,Jun 7 2022 1:48PM,239014,10,MET210987,"Methapharm, Inc.",Released
9,Jun 7 2022 1:32PM,239012,10,MET210957,"Methapharm, Inc.",Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,239011,Executing,1
42,239012,Executing,10
43,239012,Released,10
44,239014,Released,8
45,239015,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239010,NaN,NaN,1.0
239011,NaN,1.0,NaN
239012,NaN,10.0,10.0
239014,NaN,NaN,8.0
239015,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238904,0.0,0.0,1.0
238907,2.0,11.0,0.0
238909,0.0,0.0,3.0
238911,0.0,0.0,1.0
238914,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238904,0,0,1
238907,2,11,0
238909,0,0,3
238911,0,0,1
238914,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238904,0,0,1
1,238907,2,11,0
2,238909,0,0,3
3,238911,0,0,1
4,238914,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238904,,,1
1,238907,2,11,
2,238909,,,3
3,238911,,,1
4,238914,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 7 2022 1:54PM,239015,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
1,Jun 7 2022 1:48PM,239014,10,"Methapharm, Inc."
9,Jun 7 2022 1:32PM,239012,10,"Methapharm, Inc."
29,Jun 7 2022 1:28PM,239011,12,"Uniderm USA, Inc"
30,Jun 7 2022 1:16PM,239009,20,"Exact-Rx, Inc."
43,Jun 7 2022 1:16PM,239010,18,"ClearSpec, LLC"
44,Jun 7 2022 1:10PM,239007,20,Alumier Labs Inc.
60,Jun 7 2022 12:57PM,239004,10,Eywa Pharma Inc.
61,Jun 7 2022 12:49PM,239003,21,"NBTY Global, Inc."
62,Jun 7 2022 12:49PM,239002,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 7 2022 1:54PM,239015,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,1
1,Jun 7 2022 1:48PM,239014,10,"Methapharm, Inc.",,,8
2,Jun 7 2022 1:32PM,239012,10,"Methapharm, Inc.",,10,10
3,Jun 7 2022 1:28PM,239011,12,"Uniderm USA, Inc",,1,
4,Jun 7 2022 1:16PM,239009,20,"Exact-Rx, Inc.",,,13
5,Jun 7 2022 1:16PM,239010,18,"ClearSpec, LLC",,,1
6,Jun 7 2022 1:10PM,239007,20,Alumier Labs Inc.,,,16
7,Jun 7 2022 12:57PM,239004,10,Eywa Pharma Inc.,,,1
8,Jun 7 2022 12:49PM,239003,21,"NBTY Global, Inc.",,,1
9,Jun 7 2022 12:49PM,239002,21,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 1:54PM,239015,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1,,
1,Jun 7 2022 1:48PM,239014,10,"Methapharm, Inc.",8,,
2,Jun 7 2022 1:32PM,239012,10,"Methapharm, Inc.",10,10,
3,Jun 7 2022 1:28PM,239011,12,"Uniderm USA, Inc",,1,
4,Jun 7 2022 1:16PM,239009,20,"Exact-Rx, Inc.",13,,
5,Jun 7 2022 1:16PM,239010,18,"ClearSpec, LLC",1,,
6,Jun 7 2022 1:10PM,239007,20,Alumier Labs Inc.,16,,
7,Jun 7 2022 12:57PM,239004,10,Eywa Pharma Inc.,1,,
8,Jun 7 2022 12:49PM,239003,21,"NBTY Global, Inc.",1,,
9,Jun 7 2022 12:49PM,239002,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 1:54PM,239015,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1,,
1,Jun 7 2022 1:48PM,239014,10,"Methapharm, Inc.",8,,
2,Jun 7 2022 1:32PM,239012,10,"Methapharm, Inc.",10,10,
3,Jun 7 2022 1:28PM,239011,12,"Uniderm USA, Inc",,1,
4,Jun 7 2022 1:16PM,239009,20,"Exact-Rx, Inc.",13,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 1:54PM,239015,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1.0,NaN,NaN
1,Jun 7 2022 1:48PM,239014,10,"Methapharm, Inc.",8.0,NaN,NaN
2,Jun 7 2022 1:32PM,239012,10,"Methapharm, Inc.",10.0,10.0,NaN
3,Jun 7 2022 1:28PM,239011,12,"Uniderm USA, Inc",NaN,1.0,NaN
4,Jun 7 2022 1:16PM,239009,20,"Exact-Rx, Inc.",13.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 7 2022 1:54PM,239015,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1.0,0.0,0.0
1,Jun 7 2022 1:48PM,239014,10,"Methapharm, Inc.",8.0,0.0,0.0
2,Jun 7 2022 1:32PM,239012,10,"Methapharm, Inc.",10.0,10.0,0.0
3,Jun 7 2022 1:28PM,239011,12,"Uniderm USA, Inc",0.0,1.0,0.0
4,Jun 7 2022 1:16PM,239009,20,"Exact-Rx, Inc.",13.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2389838,42.0,24.0,0.0
12,477995,2.0,1.0,0.0
15,716824,45.0,35.0,0.0
16,477814,0.0,22.0,4.0
17,238911,1.0,0.0,0.0
18,477972,2.0,0.0,0.0
19,1194848,4.0,1.0,0.0
20,1911697,129.0,64.0,0.0
21,716921,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2389838,42.0,24.0,0.0
1,12,477995,2.0,1.0,0.0
2,15,716824,45.0,35.0,0.0
3,16,477814,0.0,22.0,4.0
4,17,238911,1.0,0.0,0.0
5,18,477972,2.0,0.0,0.0
6,19,1194848,4.0,1.0,0.0
7,20,1911697,129.0,64.0,0.0
8,21,716921,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,42.0,24.0,0.0
1,12,2.0,1.0,0.0
2,15,45.0,35.0,0.0
3,16,0.0,22.0,4.0
4,17,1.0,0.0,0.0
5,18,2.0,0.0,0.0
6,19,4.0,1.0,0.0
7,20,129.0,64.0,0.0
8,21,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,42.0
1,12,Released,2.0
2,15,Released,45.0
3,16,Released,0.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20,21
Status,,,,,,,,,
Completed,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
Executing,24.0,1.0,35.0,22.0,0.0,0.0,1.0,64.0,0.0
Released,42.0,2.0,45.0,0.0,1.0,2.0,4.0,129.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20,21
0,Completed,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
1,Executing,24.0,1.0,35.0,22.0,0.0,0.0,1.0,64.0,0.0
2,Released,42.0,2.0,45.0,0.0,1.0,2.0,4.0,129.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20,21
0,Completed,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
1,Executing,24.0,1.0,35.0,22.0,0.0,0.0,1.0,64.0,0.0
2,Released,42.0,2.0,45.0,0.0,1.0,2.0,4.0,129.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()